# Assignment week 5: Attention

First we import the necessary libraries and download the dataset. The dataset is split into Training and Testing. The reviews in the dataset are limited to max 250 words, and padding is added to fill in shorter reviews.

In [22]:
import keras

from keras.datasets import imdb

# Dataset variables
max_features = 10000 # vocabulary size
max_len = 250 # words per sample 

# Creating the Training and Testing sets
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Add padding to shorter reviews 
x_train = keras.utils.pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = keras.utils.pad_sequences(x_test, maxlen=max_len, padding='post')

Here I tokenized the dataset, and made a list of the word indexes. 

In [23]:
word_to_index = imdb.get_word_index()
index_to_word = dict((value, key) for (key, value) in word_to_index.items())

example_review = " ".join(index_to_word.get(i-3, "?") for i in x_train[0])
print(example_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

Here we define a custom layer to create the word vectors from the tokens. This layer takes into account the positions of the words within a sentence. 

In [24]:
from keras.layers import Layer, Embedding

class TokenAndPositionEmbedding(Layer):
    def __init__(self, seq_len, vocab_size, emb_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=emb_dim)
        self.pos_emb = Embedding(input_dim=seq_len, output_dim=emb_dim)

    def call(self, x_input):
        seq_len = keras.ops.shape(x_input)[-1]
        positions = keras.ops.arange(start=0, stop=seq_len, step=1)
        positions = self.pos_emb(positions)
        x_input = self.token_emb(x_input)
        return x_input + positions


## Model architecture

I separated the model into several blocks for clarity.

Block 1: Inputs 
- Input
- Tokenization (residual connection to Block 2)
- MultiHeadAttention

Block 2: Add & Norm 1
- Add (combines output from Tokenization and MultiHeadAttention)
- Normalization (residual connection to block 4)

Block 3: FCN
- Dense layer, 128 units (2x the second Dense) and ReLu activation
- Dropout(0.1), small dropout to improve accuracy
- Dense layer, 64 units (matching embed_dim) and no activation

Block 4: Add & Norm 2
- Add (combines output from FCN and Add & Norm 1)
- Normalization

Block 5: Outputs
- GlobalAveragePooling1D + GlobalMaxPooling1D combined
- Dropout
- Output

The model is compiled with Adam optimizer.

In [25]:
from keras.layers import Input, Dense, Concatenate,GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, Add, LayerNormalization
from keras.models import Model
from keras.layers import MultiHeadAttention

num_heads = 3 # number of attention heads
embed_dim = 96 # word embeddings dimension
key_dim = embed_dim // num_heads # query/key dimension for one head

# Inputs, tokenization and attention layer
inputs = Input(shape=(max_len,))
v = TokenAndPositionEmbedding(max_len, max_features, embed_dim)(inputs)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=0.1)(v, v, v) 

# Add & Norm 1
# Residual input from vectoring layer
x = Add()([attention_output, v])
x = LayerNormalization(epsilon=1e-6)(x)

# Fully connected feedforward layer
ff = Dense(embed_dim*2, activation="relu")(x)
ff = Dropout(0.1)(ff)
ff = Dense(embed_dim)(ff)

# Add & Norm 2
# Residual input from Add & Norm 1
x = Add()([ff, x])
x = LayerNormalization(epsilon=1e-6)(x)

# Pooling, Dropout and outputs
x = Concatenate()([
    GlobalAveragePooling1D()(x),
    GlobalMaxPooling1D()(x)
])
x = Dropout(0.3)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 250)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, 250, 96)   │    984,000 │ input_layer_6[0]… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 250, 96)   │     37,248 │ token_and_positi… │
│ (MultiHeadAttentio… │                   │            │ token_and_positi… │
│                     │                   │            │ token_and_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 250, 96)   │          0 │ multi_head_atten… │
│                     │                   │            │ token_and_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 250, 96)   │        192 │ add_10[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 250, 192)  │     18,624 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 250, 192)  │          0 │ dense_14[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 250, 96)   │     18,528 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 250, 96)   │          0 │ dense_15[0][0],   │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 250, 96)   │        192 │ add_11[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 96)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 96)        │          0 │ layer_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 192)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 192)       │          0 │ concatenate_3[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 1)         │        193 │ dropout_15[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,058,977 (4.04 MB)

 Trainable params: 1,058,977 (4.04 MB)

 Non-trainable params: 0 (0.00 B)

The model is trained for 5 epochs with batch size 32.

In [26]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 122s 150ms/step - accuracy: 0.7686 - loss: 0.4772
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 115s 147ms/step - accuracy: 0.8998 - loss: 0.2492
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 115s 147ms/step - accuracy: 0.9342 - loss: 0.1769
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 117s 150ms/step - accuracy: 0.9536 - loss: 0.1283
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 116s 149ms/step - accuracy: 0.9655 - loss: 0.0985


In [27]:
print(f'Test accuracy = {model.evaluate(x_test, y_test)[1]:.4f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.8544 - loss: 0.4292
Test accuracy = 0.8544


The final test accuracy of this model is 85,4%.